In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x))


# Crear la red a analizar
red2 = pp.create_empty_network()

# Creación de las barras del sistema
barra_1 = pp.create_bus(red2, vn_kv=110, name="Barra 1")
barra_2 = pp.create_bus(red2, vn_kv=220, name="Barra 2")
barra_1A = pp.create_bus(red2, vn_kv=220, name="Barra 1A")
barra_1B = pp.create_bus(red2, vn_kv=220, name="Barra 1B")
barra_2A = pp.create_bus(red2, vn_kv=220, name="Barra 2A")
barra_2B = pp.create_bus(red2, vn_kv=220, name="Barra 2B")
barra_3A = pp.create_bus(red2, vn_kv=220, name="Barra 3A")

# Añadir transformador entre barra 1 y 2
pp.create_transformer(red2, barra_2, barra_1, std_type="100 MVA 220/110 kV")

# Añadir el generador en la barra 1 y que dicha barra sea la referencia
pp.create_ext_grid(red2, barra_1, va_pu=1.0, name="Slack")

# Crear las líneas
pp.create_line(red2, barra_2, barra_1A, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1A")
pp.create_line(red2, barra_2, barra_1B, length_km=10.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2-1B")
pp.create_line(red2, barra_1A, barra_2A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1A-2A")
pp.create_line(red2, barra_2A, barra_3A, length_km=20.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2A-3A")
pp.create_line(red2, barra_1B, barra_2B, length_km=30.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L1B-2B")
pp.create_line(red2, barra_2B, barra_3A, length_km=15.0, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="L2B-3A")

# Crear las cargas
pp.create_load(red2, barra_1A, p_mw=30.0, q_mvar=20.0, name="Carga Barra 1A")
pp.create_load(red2, barra_2A, p_mw=52.5, q_mvar=35.0, name="Carga Barra 2A")
pp.create_load(red2, barra_3A, p_mw=22.5, q_mvar=15.0, name="Carga Barra 3A")
pp.create_load(red2, barra_1B, p_mw=15.0, q_mvar=10.0, name="Carga Barra 1B")
pp.create_load(red2, barra_2B, p_mw=90.0, q_mvar=60.0, name="Carga Barra 2B")

# Ejecutar el flujo de carga
pp.runpp(red2, algorithm='nr', numba=False, max_iteration=1000)

# Imprimir los resultados
red2.res_line.index = red2.line.name
red2.res_bus.index = red2.bus.name

print(red2.res_bus)
print(red2.res_line)

          vm_pu  va_degree      p_mw  q_mvar
name                                        
Barra 1  1.0000     0.0000 -212.3102  5.8073
Barra 2  1.0334   -14.2765    0.0000  0.0000
Barra 1A 1.0315   -14.4923   30.0000 20.0000
Barra 1B 1.0324   -14.4698   15.0000 10.0000
Barra 2A 1.0292   -14.7303   52.5000 35.0000
Barra 2B 1.0274   -14.8735   90.0000 60.0000
Barra 3A 1.0284   -14.8649   22.5000 15.0000
        p_from_mw  q_from_mvar   p_to_mw  q_to_mvar  pl_mw  ql_mvar  \
name                                                                  
L2-1A    116.8068     -22.3856 -116.5426     2.5430 0.2642 -19.8426   
L2-1B     94.2749     -37.5516  -94.0903    17.5655 0.1846 -19.9860   
L1A-2A    86.5426     -22.5430  -86.3250    -7.3809 0.2176 -29.9238   
L2A-3A    33.8250     -27.6191  -33.7786   -12.5423 0.0463 -40.1614   
L1B-2B    79.0903     -27.5655  -78.7297   -32.3478 0.3606 -59.9134   
L2B-3A   -11.2703     -27.6522   11.2786    -2.4577 0.0083 -30.1099   

        i_from_ka  i_to_ka